This code exists to open the annotations.csv file as a pandas dataframe, and convert the mm coordinates and cancer cell diameter into voxel coordinates.  For some reason, the loop took a really long time to run for me, probably because of the innefficiencies of the loop and what not, but it works.  The final dataframe are all the patients whose cancer cells get pushed out of the bounds of the CT scans (once the cells get converted to voxel coordinates).

In [ ]:
import pandas as pd
import SimpleITK as sitk
import os
from tqdm import tqdm

raw_path = 'C:/Users/572203/Documents/Data Science Bowl 2017/LUNA16/all_patients' #Hard code the location of the data LUNA16 patients.  It is convenient if all the patients are in one folder, like mine are.
cancer_annotations = pd.read_csv("C:/Users/572203/Documents/Data Science Bowl 2017/LUNA16/annotations.csv", header = 0)
test_cancer_annotations = cancer_annotations
cancer_patients = sorted(list(set(cancer_annotations.seriesuid.values)))

for fcount, i in enumerate(tqdm(cancer_patients)):
    raw_patient_path = (raw_path+'/'+str(i)+'.mhd')
    raw_slices = sitk.ReadImage(raw_patient_path)
    raw_spacing = raw_slices.GetSpacing()
    raw_origin = raw_slices.GetOrigin()
    test_cancer_annotations.loc[test_cancer_annotations.seriesuid==i,'coordX'] = (test_cancer_annotations.loc[test_cancer_annotations.seriesuid==i,'coordX'] - raw_origin[0])/raw_spacing[0]
    test_cancer_annotations.loc[test_cancer_annotations.seriesuid==i,'coordY'] = (test_cancer_annotations.loc[test_cancer_annotations.seriesuid==i,'coordY'] - raw_origin[1])/raw_spacing[1]
    test_cancer_annotations.loc[test_cancer_annotations.seriesuid==i,'coordZ'] = (test_cancer_annotations.loc[test_cancer_annotations.seriesuid==i,'coordZ'] - raw_origin[2])/raw_spacing[2]
    test_cancer_annotations.loc[test_cancer_annotations.seriesuid==i,'diameter_mm'] = test_cancer_annotations.loc[test_cancer_annotations.seriesuid==i,'diameter_mm'] / raw_spacing[0]

test_cancer_annotations.coordX = round(test_cancer_annotations.coordX)
test_cancer_annotations.coordY = round(test_cancer_annotations.coordY)
test_cancer_annotations.coordZ = round(test_cancer_annotations.coordZ)

out_of_bounds_cancer_cells = test_cancer_annotations[test_cancer_annotations.coordX<0] #don't worry, this selection captures
#all of the patients whose coordinates (either X or Y coordinates) are negative

The "out_of_bounds_cancer_cells" dataframe contains tumors, that when converted to voxel coordinates, get pushed out of the bounds of the CT scan.  As we all know, each CT scan slice are 512 * 512 pixels.  If we "open up" a CT scan slice, the pixels are numbered in the positive direction.  Basically, there are no negative pixel locations! Therefore, negative pixel locations for cancer cells are very suspect! 

This means there is an issue with either the cancer coordinates in the annotations.csv file, or the patients' metadata is incorrectly marked. Either way this requires more looking into, and maybe even excluding these patients from the training data...